In [1]:
import os
import pandas as pd
# requires dammit env
# source activate dammit
from dammit.fileio.gff3 import GFF3Parser

# All contigs quantified by salmon

In [4]:
table = pd.read_csv("/Users/johnsolk/Documents/UCDavis/Whitehead/RNAseq_15killifish/salmon_denovo_by_species/L_goodei/L_goodei_transfer_2.quant/quant.sf",sep="\t")
table.head()

,Name,Length,EffectiveLength,TPM,NumReads
0,TRINITY_DN2202_c0_g1_i1,278,70.590,0.000000,0.000000
1,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000
2,TRINITY_DN2271_c0_g1_i1,305,88.409,0.000000,0.000000
3,TRINITY_DN2271_c1_g1_i1,525,279.936,0.000000,0.000000
4,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129


In [5]:
table.shape

(385474, 5)

In [20]:
contig_names = pd.DataFrame(table.Name.str.split("_",5).tolist(),columns = ['Trinity','Contig','Component','Gene','Isoform'])

In [21]:
contig_names.head()

,Trinity,Contig,Component,Gene,Isoform
0,TRINITY,DN2202,c0,g1,i1
1,TRINITY,DN2202,c1,g1,i1
2,TRINITY,DN2271,c0,g1,i1
3,TRINITY,DN2271,c1,g1,i1
4,TRINITY,DN2268,c0,g1,i1


In [42]:
contig_names2 = pd.DataFrame()
contig_names2['Name'] = contig_names[['Trinity','Contig','Component','Gene','Isoform']].apply(lambda x: '_'.join(x),axis=1)

In [43]:
contig_names2['Gene'] = contig_names[['Trinity','Contig','Component','Gene']].apply(lambda x: '_'.join(x),axis=1)

In [44]:
contig_names2.head()

,Name,Gene
0,TRINITY_DN2202_c0_g1_i1,TRINITY_DN2202_c0_g1
1,TRINITY_DN2202_c1_g1_i1,TRINITY_DN2202_c1_g1
2,TRINITY_DN2271_c0_g1_i1,TRINITY_DN2271_c0_g1
3,TRINITY_DN2271_c1_g1_i1,TRINITY_DN2271_c1_g1
4,TRINITY_DN2268_c0_g1_i1,TRINITY_DN2268_c0_g1


In [45]:
contig_names2.shape

(385474, 2)

# Trinity "genes"

In [46]:
contig_names2['Gene'].nunique()

266317

In [48]:
table_2 = pd.merge(table,contig_names2,on='Name')

In [49]:
table_2.shape

(385474, 6)

In [50]:
table_2.head()

,Name,Length,EffectiveLength,TPM,NumReads,Gene
0,TRINITY_DN2202_c0_g1_i1,278,70.590,0.000000,0.000000,TRINITY_DN2202_c0_g1
1,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000,TRINITY_DN2202_c1_g1
2,TRINITY_DN2271_c0_g1_i1,305,88.409,0.000000,0.000000,TRINITY_DN2271_c0_g1
3,TRINITY_DN2271_c1_g1_i1,525,279.936,0.000000,0.000000,TRINITY_DN2271_c1_g1
4,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1


# Removed contigs with no expression

In [68]:
table_filt = table_2.loc[table_2['NumReads'] > 0.0]

In [69]:
table_filt.shape

(202604, 6)

In [70]:
table_filt.head()

,Name,Length,EffectiveLength,TPM,NumReads,Gene
1,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000,TRINITY_DN2202_c1_g1
4,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1
5,TRINITY_DN2201_c0_g1_i1,211,38.753,2.192735,1.000000,TRINITY_DN2201_c0_g1
7,TRINITY_DN2275_c0_g1_i1,309,91.270,0.931033,1.000000,TRINITY_DN2275_c0_g1
9,TRINITY_DN2272_c0_g1_i1,374,142.516,0.596254,1.000000,TRINITY_DN2272_c0_g1


# Trinity "genes", contigs removed with no expression

In [71]:
table_filt['Gene'].nunique()

142016

In [72]:
name = "/Users/johnsolk/Documents/UCDavis/Whitehead/gff_annotations/L_goodei.trinity_out.Trinity.fasta.dammit.gff3"
conversion = "/Users/johnsolk/Documents/UCDavis/Whitehead/dammit_conversions/L_goodei.trinity_out.Trinity.fasta.dammit.namemap.csv"
Fhet_counts = "/Users/johnsolk/Documents/UCDavis/Whitehead/Fhet_counts/L_goodei_genenames.csv"

In [73]:
annotations = GFF3Parser(filename=name).read()
annotations = annotations.dropna(subset=['Name'])

/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):
/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):


In [74]:
annotations["length"] = annotations["end"].subtract(annotations["start"], fill_value=0)

# Annotations in all contigs

In [75]:
pickonename = annotations.sort_values(by=['seqid', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','Note','database','Dbxref','start','end','length']]
pickonename = pickonename.dropna(axis=0,how="all")
print('Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):')
print(pickonename.shape)

Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(138485, 8)


In [76]:
pickonename.head()

,seqid,Name,Note,database,Dbxref,start,end,length
110,Transcript_10000,ENSXMAP00000013838,NaN,OrthoDB,NaN,1,382,381
2,Transcript_100000,ENSXMAP00000016367,NaN,OrthoDB,NaN,163,1480,1317
9,Transcript_100001,ENSXMAP00000002468,NaN,OrthoDB,NaN,367,739,372
16,Transcript_100002,ENSXMAP00000002468,NaN,OrthoDB,NaN,19,298,279
24,Transcript_100003,ENSXMAP00000017080,NaN,OrthoDB,NaN,285,1002,717


In [88]:
conversions = pd.read_csv(conversion)

In [89]:
conversions['Name'], conversions['info'] = conversions['original'].str.split(' ', 1).str
conversions = conversions[['Name','renamed']]
conversions.columns = ['Name','seqid']
conversions.head()

,Name,seqid
0,TRINITY_DN2202_c0_g1_i1,Transcript_0
1,TRINITY_DN2202_c1_g1_i1,Transcript_1
2,TRINITY_DN2271_c0_g1_i1,Transcript_2
3,TRINITY_DN2271_c1_g1_i1,Transcript_3
4,TRINITY_DN2268_c0_g1_i1,Transcript_4


In [97]:
table_filt.shape

(202604, 6)

In [105]:
merged_table = pd.merge(table_filt,conversions,on="Name")
merged_table.shape

(202604, 7)

In [92]:
merged_table = pd.merge(merged_table,pickonename,on="seqid")

# Annotations, removed contigs with no expression

In [93]:
merged_table.shape

(87271, 14)

In [99]:
merged_table.head()

,Name_x,Length,EffectiveLength,TPM,NumReads,Gene,seqid,Name_y,Note,database,Dbxref,start,end,length
0,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187,271,84
1,TRINITY_DN2223_c0_g1_i1,272,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2,272,270
2,TRINITY_DN2294_c0_g1_i1,691,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1,85,84
3,TRINITY_DN2293_c0_g1_i1,646,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395,482,87
4,TRINITY_DN73018_c0_g1_i1,425,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0,425,425


# Unique gene names in filtered contigs

In [95]:
print(len(merged_table.Name_y.unique()))

36695


# Unique Trinity "genes" in filtered contigs

In [96]:
print(len(merged_table.Gene.unique()))

43627


In [112]:
fhet = annotations[annotations['Name'].str.startswith("gi")]
fhet_filtered = fhet.query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','start','end','length']]
print("Number of unique Fhet gene names:")
print(len(fhet_filtered.Name.unique()))

Number of unique Fhet gene names:
22631


In [104]:
fhet_filtered.head()

,seqid,Name,start,end,length
25,Transcript_100003,gi|831567809|ref|XP_012733833.1| PREDICTED: my...,95,334,239
34,Transcript_100004,gi|831567809|ref|XP_012733833.1| PREDICTED: my...,0,83,83
50,Transcript_100006,gi|831503966|ref|XP_012711679.1| PREDICTED: pi...,413,433,20
68,Transcript_100007,gi|831503966|ref|XP_012711679.1| PREDICTED: pi...,419,439,20
86,Transcript_100008,gi|831567809|ref|XP_012733833.1| PREDICTED: my...,95,278,183


# Filtered expresion table, removed contigs with no expression

In [106]:
merged_table.shape

(202604, 7)

In [107]:
merged_table.head()

,Name,Length,EffectiveLength,TPM,NumReads,Gene,seqid
0,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000,TRINITY_DN2202_c1_g1,Transcript_1
1,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4
2,TRINITY_DN2201_c0_g1_i1,211,38.753,2.192735,1.000000,TRINITY_DN2201_c0_g1,Transcript_5
3,TRINITY_DN2275_c0_g1_i1,309,91.270,0.931033,1.000000,TRINITY_DN2275_c0_g1,Transcript_7
4,TRINITY_DN2272_c0_g1_i1,374,142.516,0.596254,1.000000,TRINITY_DN2272_c0_g1,Transcript_9


# *F. heteroclitus* (RefSeq) annotations, removed contigs with no expression

In [108]:
fhet_merged_table = pd.merge(merged_table,fhet_filtered,on='seqid')

In [109]:
fhet_merged_table.shape

(50975, 11)

In [116]:
fhet_merged_table.columns = ['Contig','Length','EffectiveLength','TPM','NumReads','Gene','dammit.seqid','FhetNCBIName','start.annottion','end.annotation','length.annotation']
fhet_merged_table.head()

,Contig,Length,EffectiveLength,TPM,NumReads,Gene,dammit.seqid,FhetNCBIName,start.annottion,end.annotation,length.annotation
0,TRINITY_DN2223_c0_g1_i1,272,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,gi|831524180|ref|XP_012718601.1| PREDICTED: le...,0,90,90
1,TRINITY_DN2294_c0_g1_i1,691,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,gi|831580603|ref|XP_012738233.1| PREDICTED: ab...,0,28,28
2,TRINITY_DN91698_c0_g1_i1,828,578.007,6.640536,45.169115,TRINITY_DN91698_c0_g1,Transcript_206,gi|831524518|ref|XP_012718723.1| PREDICTED: du...,26,208,182
3,TRINITY_DN91698_c0_g2_i1,243,51.396,3.027098,1.830885,TRINITY_DN91698_c0_g2,Transcript_207,gi|831524518|ref|XP_012718723.1| PREDICTED: du...,26,79,53
4,TRINITY_DN91660_c0_g1_i1,520,275.173,0.309218,1.001326,TRINITY_DN91660_c0_g1,Transcript_226,gi|831481653|ref|XP_012728360.1| PREDICTED: zi...,0,56,56


# Unique *F. heteroclitus* (RefSeq) annotations, removed contigs with no expression

In [117]:
print(len(fhet_merged_table.FhetNCBIName.unique()))

20674
